<a href="https://colab.research.google.com/github/yamadashamoji/tools/blob/main/102_inventory_forcastingv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title 在庫予測アプリ（Google Colab用）

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import io
from IPython.display import display, clear_output
from google.colab import files
import ipywidgets as widgets

# --- UI定義 ---
start_date = widgets.DatePicker(description='開始日')
end_date = widgets.DatePicker(description='終了日')
product_names = widgets.Text(value='製品A,製品B', description='製品名')
initial_inventory = widgets.IntSlider(value=500, min=0, max=1000, step=10, description='初期在庫')
predict_days = widgets.IntSlider(value=7, min=1, max=30, step=1, description='予測日数')
run_button = widgets.Button(description='在庫予測を実行', button_style='success')
download_button = widgets.Button(description='CSVをダウンロード', button_style='info')

# 出力エリア
output = widgets.Output()

# --- 共通関数定義 ---
def calculate_inventory(start_val, production, shipment):
    inventory = []
    current = start_val
    for p, s in zip(production, shipment):
        current = max(0, current + p - s)
        inventory.append(current)
    return inventory

def validate_inputs():
    s = start_date.value
    e = end_date.value
    if s is None or e is None:
        return False, "⚠️ 開始日と終了日を選択してください。"
    if e < s:
        return False, "⚠️ 終了日は開始日より後にしてください。"
    names = [n.strip() for n in product_names.value.split(',') if n.strip()]
    if not names:
        return False, "⚠️ 製品名を1つ以上入力してください。"
    return True, (s, e, names)

def generate_product_data(product, start, end, init_inv, future_days):
    date_range = pd.date_range(start, end)
    n = len(date_range)
    production = np.random.normal(100, 10, n).astype(int)
    shipment = np.random.normal(90, 10, n).astype(int)
    inventory = calculate_inventory(init_inv, production, shipment)

    df = pd.DataFrame({
        'Product': product,
        'Date': date_range,
        'Production': production,
        'Shipment': shipment,
        'Inventory': inventory
    })

    avg_prod = int(df['Production'][-7:].mean())
    avg_ship = int(df['Shipment'][-7:].mean())
    last_inv = df['Inventory'].iloc[-1]

    future_dates = pd.date_range(end + datetime.timedelta(days=1), periods=future_days)
    f_prod = [avg_prod] * future_days
    f_ship = [avg_ship] * future_days
    f_inv = calculate_inventory(last_inv, f_prod, f_ship)

    future_df = pd.DataFrame({
        'Product': product,
        'Date': future_dates,
        'Production': f_prod,
        'Shipment': f_ship,
        'Inventory': f_inv
    })

    return pd.concat([df, future_df])

def generate_inventory_forecast(start, end, products, init_inv, future_days):
    return pd.concat([
        generate_product_data(p, start, end, init_inv, future_days)
        for p in products
    ])

def plot_inventory(df, product, today):
    plt.figure(figsize=(10, 4))
    plt.plot(df['Date'], df['Inventory'], label=product)
    plt.axvline(today, color='gray', linestyle='--', label='今日')
    plt.title(f'{product}の在庫予測')
    plt.xlabel('日付')
    plt.ylabel('在庫数')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

forecast_data = None

def on_run_clicked(b):
    with output:
        clear_output()
        global forecast_data

        valid, result = validate_inputs()
        if not valid:
            print(result)
            return

        s, e, names = result

        try:
            forecast_data = generate_inventory_forecast(
                s, e, names, initial_inventory.value, predict_days.value)

            for product in names:
                df = forecast_data[forecast_data['Product'] == product]
                today = pd.to_datetime(e)
                plot_inventory(df, product, today)

            display(download_button)

        except Exception as ex:
            print(f"エラーが発生しました: {ex}")

def on_download_clicked(b):
    if forecast_data is not None:
        buffer = io.StringIO()
        forecast_data.to_csv(buffer, index=False)
        buffer.seek(0)
        files.download('multi_inventory_forecast.csv')
    else:
        with output:
            clear_output()
            print("⚠️ データがまだ生成されていません。")

# --- イベント紐づけ ---
run_button.on_click(on_run_clicked)
download_button.on_click(on_download_clicked)

# --- UI表示 ---
ui = widgets.VBox([
    widgets.HBox([start_date, end_date]),
    product_names,
    initial_inventory,
    predict_days,
    run_button,
    output
])

display(ui)

---

# 📄 在庫予測アプリ 仕様書（Google Colab版）

---

## 1. 概要

本アプリは、製造業における複数製品の在庫管理を支援するための予測ツールである。Google Colab上で動作し、ダミーデータを使用して各製品の生産量・出荷量をもとに在庫推移を計算・可視化し、将来の在庫量を予測する。生成データはCSVとしてダウンロード可能。

---

## 2. 対象ユーザー

- 製造業の在庫担当者
- データ分析の学習者
- Python／Colabを使った業務自動化に興味があるユーザー

---

## 3. 主な機能

| 機能                     | 説明                                                                 |
|--------------------------|----------------------------------------------------------------------|
| UIフォーム入力           | 日付範囲、製品リスト、初期在庫、予測期間を入力可能なインターフェース        |
| ダミーデータ生成         | 各製品について、生産量・出荷量をランダムに生成（正規分布ベース）            |
| 在庫計算                 | 日次で「在庫＝前日在庫＋生産－出荷」を計算し、マイナスはゼロとして扱う         |
| 在庫予測（未来）         | 直近7日間の平均値を使って未来日数分を予測                                    |
| グラフ描画               | 各製品の在庫推移（過去＋未来）を折れ線グラフで表示                           |
| CSV出力・ダウンロード    | 生成された全データを1ファイルにまとめてCSVとして保存＆ダウンロード可能         |

---

## 4. 使用技術

| 項目             | 技術内容                         |
|------------------|----------------------------------|
| 実行環境         | Google Colab                     |
| 言語             | Python 3                         |
| 可視化           | Matplotlib                       |
| UIインターフェース | ipywidgets（DatePicker, Text, Slider, Button）|
| データ出力       | pandas, google.colab.files       |

---

## 5. 入力項目（UIフォーム）

| 入力項目             | 型          | 説明                                               |
|----------------------|-------------|----------------------------------------------------|
| 開始日               | DatePicker  | 生産・出荷データの開始日                           |
| 終了日               | DatePicker  | 生産・出荷データの終了日                           |
| 製品名リスト         | Text        | カンマ区切りで複数入力（例：製品A,製品B）          |
| 初期在庫             | IntSlider   | 各製品共通での初期在庫（全て同じ値を設定）         |
| 予測日数             | IntSlider   | 将来予測の対象日数（1～30日）                      |
| 実行ボタン           | Button      | データ生成と可視化・ダウンロードを実行             |

---

## 6. 出力内容

### 表形式（CSV）
| Product | Date       | Production | Shipment | Inventory |
|---------|------------|------------|----------|-----------|
| 製品A   | 2025-04-01 | 101        | 91       | 510       |
| …       | …          | …          | …        | …         |

- 全製品・全日付分のデータ（過去＋未来）
- ファイル名：`multi_inventory_forecast.csv`

### グラフ
- 折れ線グラフ（各製品の在庫推移）
- グレーの縦線で「今日（実績と予測の境界）」を表示

---

## 7. 想定される今後の拡張機能（案）

| 拡張案                           | 説明                                               |
|----------------------------------|----------------------------------------------------|
| 製品ごとに個別初期在庫を設定     | 製品Aは500、製品Bは300などの個別設定対応             |
| 出荷・生産パターンの調整         | 曜日効果・季節性の追加など                         |
| PDFや画像としての出力            | 結果グラフやテーブルをPDF/PNGで保存可能に          |
| 統計モデルや機械学習での予測     | 単純平均ではなく、ARIMAやLSTMなどの予測手法に対応   |
| 入力データのCSVアップロード対応   | 実データベースの予測にも活用可能に                 |

---

## 8. サンプル画面（UI例）

```
📅 開始日：[ 2025-04-01 ]   終了日：[ 2025-04-30 ]
🧪 製品名（,区切り）：[ 製品A,製品B ]
📦 初期在庫： [ 500 ]
🔮 予測日数： [ 7 ]
✅ [ 在庫予測を実行 ] ボタン
```

---

## UI構成図

```
┌────────────────────────────────────────────┐
│ 📅 開始日： [DatePicker]　　終了日： [DatePicker]         │
│ 🧪 製品名（,区切り）：[ Text ]                              │
│ 📦 初期在庫： [ IntSlider ]                                │
│ 🔮 予測日数： [ IntSlider ]                                │
│ ✅ [ 在庫予測を実行 ]（Button）                            │
├────────────────────────────────────────────┤
│ 📊 折れ線グラフ（在庫推移：実績＋予測）                     │
├────────────────────────────────────────────┤
│ 📁 [ CSVダウンロードボタン ]                                │
└────────────────────────────────────────────┘
```

---

## データ構成図

```
+----------+------------+-------------+-----------+------------+
| Product  |    Date    | Production  | Shipment  | Inventory  |
+----------+------------+-------------+-----------+------------+
| 製品A    | 2025-04-01 |    100      |    90     |    510     |
| 製品A    | 2025-04-02 |    102      |    92     |    520     |
|   ⋮      |     ⋮      |     ⋮       |     ⋮     |     ⋮      |
| 製品B    | 2025-04-01 |    98       |    89     |    500     |
|   ⋮      |     ⋮      |     ⋮       |     ⋮     |     ⋮      |
| 製品A    | 未来予測日 |  (平均)     |  (平均)   |  計算値     |
| 製品B    | 未来予測日 |  (平均)     |  (平均)   |  計算値     |
+----------+------------+-------------+-----------+------------+

※ Inventory = 前日在庫 + Production - Shipment（下限0）
※ Productごとに日付データが展開されており、未来予測も含まれる

```

---